In [5]:
import os
from os.path import join, dirname
from dotenv import load_dotenv

dotenv_path = join(dirname('__file__'), '.env')

load_dotenv(dotenv_path)

DATALAKE_PATH = os.environ.get("DATALAKE_PATH")
ROOT_PATH = os.environ.get("ROOT_PATH")
SOURCE_PATH = os.environ.get("SOURCE_PATH")

SUB_PATH = 'tuh_seizures/processed/features/dev'

In [40]:
import pandas as pd 
from scipy.io import loadmat
import numpy as np
from joblib import Parallel, delayed

In [11]:
files = pd.read_csv(f'{DATALAKE_PATH}/{SUB_PATH}/files.txt', header=None)[0].to_list()

In [39]:
def pipe(f:str) -> any:
    cols = ['kur_log','skew_log','var_log']

    d = loadmat(f'{DATALAKE_PATH}/{SUB_PATH}/{f}')
    df = pd.DataFrame(data=[d['kur_log'][0],d['skew_log'][0],d['var_log'][0]], index=cols).T
    df['label'] = np.repeat(d['label'][0],len(df))

    return df



In [41]:
results = Parallel(n_jobs=-1)(delayed(pipe)(f) for f in files)

In [44]:
pd.concat(results).to_parquet('../features/dev_features_dataset.parquet',index=False)